In [1]:
from inference import get_model
import supervision as sv
import os
from pathlib import Path
from tqdm import tqdm
import time

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


In [2]:
api_key="Ri8TOlA44FZ62FfabnTt"
# Initialize the models
pose_model = get_model(model_id="yolov8x-pose-1280", api_key=api_key)
ball_model = get_model(model_id="padel_ball/3", api_key=api_key)

In [3]:
def process_image(image_path, model, process_type):
    results = model.infer(str(image_path))[0]
    
    if process_type == 'pose':
        return sv.KeyPoints.from_inference(results)
    elif process_type == 'ball':
        detections = sv.Detections.from_inference(results)
        return [(box.tolist(), conf) for box, conf in zip(detections.xyxy, detections.confidence)]

def process_folders(base_path='assets', models=None):
    if models is None:
        models = {'pose': pose_model, 'ball': ball_model}
    
    data = {model_name: {} for model_name in models}
    
    base_path = Path(base_path)
    
    # Count total PNG files in immediate subfolders
    total_files = sum(len(list(subfolder.glob('*.png'))) 
                      for subfolder in base_path.iterdir() 
                      if subfolder.is_dir())
    
    print(f"Total PNG files to process: {total_files}")
    
    with tqdm(total=total_files, desc="Processing images") as pbar:
        for subfolder in base_path.iterdir():
            if subfolder.is_dir():
                folder_name = subfolder.name
                for model_name in models:
                    data[model_name][folder_name] = {}
                
                for png_file in subfolder.glob('*.png'):
                    frame_number = int(png_file.stem)
                    for model_name, model in models.items():
                        try:
                            results = process_image(png_file, model, model_name)
                            data[model_name][folder_name][frame_number] = results
                        except Exception as e:
                            print(f"Error processing {png_file} with {model_name} model: {str(e)}")
                    pbar.update(1)  # Update once per image
    
    return data

In [4]:
# Run the processing
all_data = process_folders()

Total PNG files to process: 4320


Processing images: 100%|██████████| 4320/4320 [58:34<00:00,  1.23it/s] 


In [5]:
# Example of how to access the data:
# all_data['pose']['1'][713] would give you the pose keypoints for frame 713 in folder 1
# all_data['ball']['1'][713] would give you the ball bounding boxes and confidences for frame 713 in folder 1

# Print some results to verify
for model_name in all_data:
    for folder in all_data[model_name]:
        print(f"{model_name} results for folder {folder}:")
        print(f"Number of frames processed: {len(all_data[model_name][folder])}")
        if all_data[model_name][folder]:
            first_frame = next(iter(all_data[model_name][folder]))
            print(f"Example result for frame {first_frame}:")
            print(all_data[model_name][folder][first_frame])
        print()

pose results for folder 1:
Number of frames processed: 1896
Example result for frame 3975:
KeyPoints(xy=array([[[1199.,  693.],
        [1199.,  683.],
        [1201.,  682.],
        [1152.,  691.],
        [1234.,  695.],
        [1099.,  777.],
        [1229.,  820.],
        [1029.,  866.],
        [1332.,  935.],
        [1043.,  885.],
        [1411.,  829.],
        [1103., 1008.],
        [1184., 1026.],
        [1167., 1047.],
        [1274., 1041.],
        [1168., 1051.],
        [1255., 1058.]],

       [[1239.,  318.],
        [1247.,  314.],
        [1235.,  314.],
        [1260.,  316.],
        [1236.,  315.],
        [1260.,  337.],
        [1247.,  336.],
        [1264.,  370.],
        [1247.,  368.],
        [1261.,  400.],
        [1244.,  395.],
        [1249.,  396.],
        [1243.,  395.],
        [1238.,  442.],
        [1251.,  440.],
        [1236.,  487.],
        [1257.,  483.]]], dtype=float32), class_id=array([0, 0]), confidence=array([[0.08766675, 0.025

In [6]:
import pickle
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f"Data loaded from {filename}")
    return data

In [13]:

# Save the data
save_data(all_data, 'all_data.pkl')

Data saved to all_data.pkl


In [14]:
copy = load_data('all_data.pkl')
# remove folder "4 from all_data ball and pose
del copy['ball']['4']
del copy['pose']['4']
# Save the data
save_data(copy, 'all_data.pkl')



Data loaded from all_data.pkl
Data saved to all_data.pkl
